<a href="https://colab.research.google.com/github/MichaelGuaca/Proyecto-IA/blob/main/99_modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importar librerías necesarias
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


In [2]:
#IMPORTAR DATOS DE KAGGLE A COLAB
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 60% 12.0M/20.1M [00:00<00:00, 122MB/s]
100% 20.1M/20.1M [00:00<00:00, 146MB/s]


In [3]:
#DESCOMPRIMIR ARCHIVO
!unzip udea-ai4eng-20242.zip

Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [4]:
#CARGAR DATOS TRAIN CON PANDAS
train_data = pd.read_csv("train.csv")

In [5]:
#Explorar datos
print(train_data.info())
print(train_data['RENDIMIENTO_GLOBAL'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 12 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              692500 non-null  int64 
 1   PERIODO                         692500 non-null  int64 
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object
 7   FAMI_TIENEINTERNET              665871 non-null  object
 8   FAMI_EDUCACIONPADRE             669322 non-null  object
 9   FAMI_EDUCACIONMADRE             668836 non-null  object
 10  ESTU_PAGOMATRICULAPROPIO        686002 non-null  object
 11  RENDIMIENTO_GLOBAL              692500 non-null  object
dtypes: int64(2), object(10)
memory

In [6]:
#Preprocesamiento de datos
#Manejar valores faltantes (rellenando con una categoría específica para columnas categóricas)
for col in train_data.columns:
    if train_data[col].dtype == 'object':
        train_data[col].fillna('Desconocido', inplace=True)
    else:
        train_data[col].fillna(train_data[col].median(), inplace=True)

<ipython-input-6-73dd2afa7a95>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data[col].fillna(train_data[col].median(), inplace=True)
<ipython-input-6-73dd2afa7a95>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

In [7]:
#Codificar variables categóricas usando LabelEncoder
categorical_cols = train_data.select_dtypes(include='object').columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    label_encoders[col] = le

In [8]:
#Separar características (X) y objetivo (y)
X = train_data.drop(['RENDIMIENTO_GLOBAL', 'ID', 'PERIODO'], axis=1)
y = train_data['RENDIMIENTO_GLOBAL']

In [42]:
#Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)

In [43]:
# Entrenar modelo XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:11:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [45]:
# Evaluar el modelo
xgb_pred = xgb_model.predict(X_val)
xgb_accuracy = accuracy_score(y_val, xgb_pred)
print(f"XGBoost Accuracy: {xgb_accuracy}")
print(classification_report(y_val, xgb_pred, target_names=label_encoders["RENDIMIENTO_GLOBAL"].classes_))


XGBoost Accuracy: 0.4291898916967509
              precision    recall  f1-score   support

        alto       0.55      0.62      0.58     87905
        bajo       0.46      0.54      0.50     86722
  medio-alto       0.33      0.27      0.30     85543
  medio-bajo       0.32      0.27      0.30     86080

    accuracy                           0.43    346250
   macro avg       0.41      0.43      0.42    346250
weighted avg       0.42      0.43      0.42    346250



In [60]:
print("Accuracy:", accuracy_score(y_val, xgb_pred))

Accuracy: 0.4291898916967509


In [61]:
# Assuming you want to make predictions for the entire train_data
features_used_in_training = xgb_model.get_booster().feature_names # Get the features used during training
xgb_pred_all = xgb_model.predict(train_data[features_used_in_training]) # Use only those features for prediction

#Guardar CSV
submission = pd.DataFrame({'ID': train_data["ID"], 'RENDIMIENTO_GLOBAL': xgb_pred_all})
submission['RENDIMIENTO_GLOBAL'] = label_encoders["RENDIMIENTO_GLOBAL"].inverse_transform(submission['RENDIMIENTO_GLOBAL'])
submission.to_csv("submission_example.csv", index=False)
print(submission)

            ID RENDIMIENTO_GLOBAL
0       904256               alto
1       645256               bajo
2       308367         medio-alto
3       470353         medio-alto
4       989032         medio-bajo
...        ...                ...
692495   25096         medio-alto
692496  754213         medio-bajo
692497  504185               bajo
692498  986620               bajo
692499  933374               alto

[692500 rows x 2 columns]


In [59]:
!kaggle competitions submit -c udea-ai4eng-20242 -f /content/submission.csv -m "Ultima Entrega"

100% 9.49M/9.49M [00:00<00:00, 37.2MB/s]
Successfully submitted to UDEA/ai4eng 20242 - Pruebas Saber Pro Colombia